<a href="https://colab.research.google.com/github/arunraja-hub/Preference_Extraction/blob/master/PreferenceNN_compiled.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/arunraja-hub/Preference_Extraction.git





Cloning into 'Preference_Extraction'...
remote: Enumerating objects: 649, done.
remote: Counting objects: 100% (649/649), done.
remote: Compressing objects: 100% (633/633), done.
remote: Total 649 (delta 85), reused 521 (delta 15), pack-reused 0
Receiving objects: 100% (649/649), 21.38 MiB | 9.19 MiB/s, done.
Resolving deltas: 100% (85/85), done.


In [2]:
from google.colab import files
src = list(files.upload().values())[0]
open('imports_data.py','wb').write(src)
import imports_data
# from imports_data.py import *
# EDA Imports
import pandas as pd
import seaborn as sns
import joblib


sns.set(style="white")

Saving imports_data.py to imports_data.py


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
all_raw_data = imports_data.all_load_data("Preference_Extraction/data/simple_env_1/")
len(all_raw_data)

475

In [0]:
activations = []
observations = []
preferences = []

In [5]:
print(all_raw_data[0].observation.shape)
print(all_raw_data[0].policy_info["activations"].shape)
print(len(all_raw_data[0].policy_info["satisfaction"].as_list()))

(50, 14, 16, 5)
(50, 64)
50


In [0]:
# Load up data into arrays
import numpy as np 
for data in all_raw_data:
    for i in range(data.observation.shape[0]):
        observations.append(np.copy(data.observation[i]))
        activations.append(np.copy(data.policy_info["activations"][i]))
        preferences.append(data.policy_info['satisfaction'].as_list()[i])

observations = np.array(observations)
activations = np.array(activations)
preferences = np.array(preferences)

deep_NN_without_regularization.ipynb

In [7]:
from lucid.misc.channel_reducer import ChannelReducer
%tensorflow_version 1.x

!pip install tf-agents==0.3.0
!pip uninstall tensorflow-probability -y
!pip install tensorflow-probability==0.7.0

import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tf_agents.trajectories.time_step import TimeStep
from tf_agents.specs.tensor_spec import TensorSpec
from tf_agents.specs.tensor_spec import TensorSpec
from tf_agents.specs.tensor_spec import BoundedTensorSpec
from tf_agents.networks import q_network

import lucid.modelzoo.vision_models as models


TensorFlow 1.x selected.
     |████████████████████████████████| 839kB 2.8MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
  Found existing installation: gin-config 0.3.0
    Uninstalling gin-config-0.3.0:
      Successfully uninstalled gin-config-0.3.0
Uninstalling tensorflow-probability-0.7.0:
  Successfully uninstalled tensorflow-probability-0.7.0
     |████████████████████████████████| 983kB 2.8MB/s 
  Found existing installation: tensorflow-probability 0.10.0rc0
    Uninstalling tensorflow-probability-0.10.0rc0:
      Successfully uninstalled tensorflow-probability-0.10.0rc0


In [0]:
cpt_name = "Preference_Extraction/model_ckpt"


In [9]:
tf.reset_default_graph()
input_shape = [14, 16, 5]

my_input = tf.placeholder(tf.float32, shape=[None] + input_shape, name="my_input")
q_vals = q_network.QNetwork(input_tensor_spec=TensorSpec(shape=(14, 16, 5)), action_spec=BoundedTensorSpec((), tf.int32, 0, 2), conv_layer_params = [[16, 3, 1], [32, 3, 2]], fc_layer_params = [64])(my_input)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [10]:
cpt_var_info = tf.compat.v1.train.list_variables(cpt_name)
cpt_var_info = [var for var in cpt_var_info if (("bias" in var[0]) or ("kernel" in var[0])) and not ("OPTIMIZER" in var[0]) and not ("_target_q_network" in var[0])]
shape_to_cpt_var_name = {tuple(var[1]): var[0] for var in cpt_var_info}

current_vars = tf.get_collection(tf.GraphKeys.VARIABLES)
shape_to_current_var_name = {tuple(var.get_shape().as_list()): var.name[:-2] for var in current_vars}

var_name_to_prev_var_name = {}
for shape in shape_to_current_var_name:
  var_name_to_prev_var_name[shape_to_current_var_name[shape]] = shape_to_cpt_var_name[shape]

Instructions for updating:
Use `tf.GraphKeys.GLOBAL_VARIABLES` instead.


In [0]:
tf.train.warm_start(cpt_name, var_name_to_prev_var_name=var_name_to_prev_var_name)
init_op = tf.global_variables_initializer()

In [0]:
conv_activations = []
conv_activation_tensor = tf.get_default_graph().get_tensor_by_name(
    "QNetwork/EncodingNetwork/EncodingNetwork/conv2d/Relu:0")

with tf.Session() as sess:
  sess.run(init_op)

  for img in observations:
    conv_act = sess.run(conv_activation_tensor, {my_input: np.array([img])})[0]
    conv_activations.append(conv_act)

conv_activations = np.array(conv_activations)

In [13]:
print('Obs.shape', observations.shape)
print('ConvAct.shape', conv_activations.shape)
print('DensAct.shape', activations.shape)
print('Pref.shape', preferences.shape)

Obs.shape (23750, 14, 16, 5)
ConvAct.shape (23750, 12, 14, 16)
DensAct.shape (23750, 64)
Pref.shape (23750,)


In [14]:
import joblib
joblib.dump(observations, 'obs.pkl')
joblib.dump(conv_activations, 'convAct.pkl')
joblib.dump(activations, 'densAct.pkl')
joblib.dump(preferences, 'pref.pkl')

['pref.pkl']

In [0]:
observations = joblib.load('obs.pkl')
conv_activations = joblib.load('convAct.pkl')
activations = joblib.load('densAct.pkl')
preferences = joblib.load('pref.pkl')

In [0]:
xs_raw = []
xs = []

for conv_activation, activation in zip(conv_activations, activations):
  xs_raw.append([conv_activation, [[activation]]])

for element in xs_raw:
  temp = []
  for subelement in element:
    for subsubelement in subelement:
      for subsubsubelement in subsubelement:
        for subsubsubsubelement in subsubsubelement:
          temp.append(subsubsubsubelement)
  temp = np.array(temp)
  xs.append(temp)
xs = np.array(xs)
ys = (preferences > -6).astype(int)

In [17]:
xs.shape

(23750, 2752)

In [0]:
def get_val_auc(logs):
      for key in logs:
        if key.startswith('val_auc'):
          return logs[key]

class BestStats(tf.keras.callbacks.Callback):
  """A callback to keep track of the best val accuracy and auc seen so far."""
  def on_train_begin(self, logs):
      self.bestMetric = -float('inf')
      self.bestLogs = None
      self.bestTrain = -float('inf')
      self.num_epochs = 0

  def on_epoch_end(self, epoch, logs):
    self.num_epochs += 1
    self.bestTrain = max(self.bestTrain, logs.get('accuracy'))

    val_accuracy = logs.get('val_accuracy')
    if val_accuracy == None:
      return 

    val_auc = get_val_auc(logs)
    
    metric = (val_accuracy + val_auc) / 2.0

    if metric > self.bestMetric:
      self.bestMetric = metric
      self.bestLogs = logs

In [0]:
# try a different model that doesn't use convolutions and Flattens() the input at first

def get_model(reg_amount, drop_rate, reduction_alg, n_components, model_name):
  del reduction_alg, n_components
  if model_name=='deepNNwoReg':

    model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(512, activation="relu"),
      tf.keras.layers.Dropout(drop_rate),
      tf.keras.layers.Dense(128, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      tf.keras.layers.Dense(1, activation='sigmoid'),
    ])
  if model_name=='deepNNwithReg':


    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(512, activation="relu"),
        tf.keras.layers.Dropout(drop_rate),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l2(reg_amount)),
      ])
  if model_name=='conv':
    model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 2, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(reg_amount), input_shape=xs.shape[1:]),
    tf.keras.layers.Conv2D(16, 1, activation='relu', strides=1, kernel_regularizer=tf.keras.regularizers.l2(reg_amount)),
    tf.keras.layers.AveragePooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(drop_rate),
    tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l2(reg_amount)),
  ])



  model.compile(optimizer=tf.keras.optimizers.Adam(.01),
                loss='binary_crossentropy',
                metrics=['accuracy',
                        tf.keras.metrics.AUC()
                        ],
                )
  return model


In [20]:
use_activations = False

drop_rate=[[.0,],[.2,]]
model_names=[['deepNNwoReg'],['deepNNwithReg'], ['conv']]
all_hparam_possibilities=[]
for d in drop_rate:
  for m in model_names:
    dict_entry={"drop_rate": d, "reg_amount": [.0], 'reduction_alg': [None], 'n_components': [None], 'model_name': m}
    all_hparam_possibilities.append(dict_entry)


print(all_hparam_possibilities)
# all_hparam_possibilities = [
#     {"drop_rate": [.0,], "reg_amount": [.0], 'reduction_alg': [None], 'n_components': [None], 'model_name': ['deepNNwoReg']},
#     {"drop_rate": [.2,], "reg_amount": [.0], 'reduction_alg': [None], 'n_components': [None],'model_name': ['deepNNwoReg']},
#     {"drop_rate": [.2,], "reg_amount": [.0], 'reduction_alg': [None], 'n_components': [None],'model_name': ['deepNNwithReg']},
#     {"drop_rate": [.2,], "reg_amount": [.0], 'reduction_alg': [None], 'n_components': [None],'model_name': ['deepNNwithReg']}]

[{'drop_rate': [0.0], 'reg_amount': [0.0], 'reduction_alg': [None], 'n_components': [None], 'model_name': ['deepNNwoReg']}, {'drop_rate': [0.0], 'reg_amount': [0.0], 'reduction_alg': [None], 'n_components': [None], 'model_name': ['deepNNwithReg']}, {'drop_rate': [0.0], 'reg_amount': [0.0], 'reduction_alg': [None], 'n_components': [None], 'model_name': ['conv']}, {'drop_rate': [0.2], 'reg_amount': [0.0], 'reduction_alg': [None], 'n_components': [None], 'model_name': ['deepNNwoReg']}, {'drop_rate': [0.2], 'reg_amount': [0.0], 'reduction_alg': [None], 'n_components': [None], 'model_name': ['deepNNwithReg']}, {'drop_rate': [0.2], 'reg_amount': [0.0], 'reduction_alg': [None], 'n_components': [None], 'model_name': ['conv']}]


In [21]:
import joblib
import concurrent.futures
import itertools
import os
import pickle
import random
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import PIL
from PIL import Image
import scipy
from scipy import ndimage
from sklearn import metrics
from sklearn.utils import shuffle
import io
import collections

%tensorflow_version 2.x
import tensorflow as tf

TensorFlow is already loaded. Please restart the runtime to change versions.


In [22]:

num_train = 50
num_val = 1000
epochs = 400
print("use_activations:", use_activations, "num_train:", num_train, "epochs", epochs)
if num_train > 50:
  print("More than 50 train data!!!!!!!!")

# each item in all_hparam_possibilities specifies valid hyper params to try. Put parameters that don't make sense together in separate lists.

hparam_combinations = []
for hparam_possibilities in all_hparam_possibilities:
  hparam_keys, hparam_values = zip(*hparam_possibilities.items())
  hparam_combinations.extend([dict(zip(hparam_keys, v)) for v in itertools.product(*hparam_values)])
random.shuffle(hparam_combinations)
print("len(hparam_combinations)", len(hparam_combinations), "hparam_combinations", hparam_combinations)

def modify_x_for_reduce(xs):
  reshaped_x = np.reshape(xs, [xs.shape[0], -1])
  # Make everything positive because some reductions don't work with negatives.
  reshaped_x -= np.min(reshaped_x)
  return reshaped_x

def unsup_exstract(xs, reg_amount, drop_rate, layer_sizes, reduction_alg, n_components):
  del reg_amount, drop_rate, layer_sizes

  print("Using unsupervised feature extraction.")

  dim_reduct_model = ChannelReducer(reduction_alg=reduction_alg, n_components=n_components)
  xs = dim_reduct_model.fit_transform(modify_x_for_reduce(xs))
  return xs

def train_best_logs(xs, ys, num_val, do_summary, hparams, get_model):
  """Trains the model and retruns the logs of the best epoch. randomly splits the train and val data before training."""
  tf.keras.backend.clear_session()
  model = get_model(**hparams)
  xs, ys = shuffle(xs, ys)

  xs_val = xs[num_train:num_train+num_val]
  ys_val = ys[num_train:num_train+num_val]
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, verbose=0)
  best_stats = BestStats()
  model.fit(xs[:num_train], ys[:num_train], epochs=epochs, batch_size=256, validation_freq=1, callbacks=[best_stats, early_stopping], validation_data=(xs_val, ys_val), verbose=0)
  if do_summary:
    model.summary()
    print("best train accuracy:", best_stats.bestTrain)
    print("Number of epochs:", best_stats.num_epochs)
  return best_stats.bestLogs

def multiple_train_ave(hparams):
  """Trains the model multiple times with the same parameters and returns the average metrics"""
  start = time.time()
  all_val_auc = []
  all_val_accuracy = []

  if hparams['reduction_alg'] != None:
    xs_for_train = unsup_exstract(xs, **hparams)
  else:
    xs_for_train = xs

  do_summary = True
  for i in range(5):
    logs = train_best_logs(xs_for_train, ys, num_val, do_summary, hparams, get_model)
    all_val_auc.append(get_val_auc(logs))
    all_val_accuracy.append(logs.get('val_accuracy'))
    do_summary = False 

  mean_val_auc = np.mean(all_val_auc)
  mean_val_accuracy = np.mean(all_val_accuracy)
  metric = (mean_val_auc + mean_val_accuracy) / 2.0
  print_data = ("mean_val_auc", mean_val_auc, "mean_val_accuracy", mean_val_accuracy, "metric", metric, "val_auc_std", np.std(all_val_auc), "val_accuracy_std", np.std(all_val_accuracy))

  end = time.time()
  print("Seconds per hyperparam config", end - start)
  # GPU: ('Seconds per hyperparam config', 16.970870971679688)

  return metric, print_data

best_metric = -float('inf')

run_num = 0
for hparams in hparam_combinations:
  print("hparams", hparams)

  metric, print_data = multiple_train_ave(hparams)

  print(print_data)
  if metric > best_metric:
    best_metric = metric
    best_print_data = print_data
    best_hparams = hparams

  run_num += 1
  print("fract done", run_num/float(len(hparam_combinations)))
  print
  print("==============================================================================================")
  print
  sys.stdout.flush()

  print("best_hparams", best_hparams)
  print("best results", best_print_data)
  print("Retraining on the best_hparams to make sure we didn't just get good results by random chance.")

  _, print_data = multiple_train_ave(best_hparams)
  print("Result of retrain on the best hyperparameters", print_data)

use_activations: False num_train: 50 epochs 400
len(hparam_combinations) 6 hparam_combinations [{'drop_rate': 0.0, 'reg_amount': 0.0, 'reduction_alg': None, 'n_components': None, 'model_name': 'deepNNwoReg'}, {'drop_rate': 0.2, 'reg_amount': 0.0, 'reduction_alg': None, 'n_components': None, 'model_name': 'conv'}, {'drop_rate': 0.0, 'reg_amount': 0.0, 'reduction_alg': None, 'n_components': None, 'model_name': 'conv'}, {'drop_rate': 0.2, 'reg_amount': 0.0, 'reduction_alg': None, 'n_components': None, 'model_name': 'deepNNwithReg'}, {'drop_rate': 0.0, 'reg_amount': 0.0, 'reduction_alg': None, 'n_components': None, 'model_name': 'deepNNwithReg'}, {'drop_rate': 0.2, 'reg_amount': 0.0, 'reduction_alg': None, 'n_components': None, 'model_name': 'deepNNwoReg'}]
hparams {'drop_rate': 0.0, 'reg_amount': 0.0, 'reduction_alg': None, 'n_components': None, 'model_name': 'deepNNwoReg'}


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


TypeError: ignored